In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import autokeras as ak
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Using TensorFlow backend


In [2]:
model = tensorflow.keras.models.load_model('./structured_data_classifier/best_model')
model.summary()

2023-12-04 12:12:47.877735: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-12-04 12:12:47.877784: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-04 12:12:47.877787: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-04 12:12:47.878076: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-04 12:12:47.878426: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-12-04 12:13:19.784899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1776)]            0         
                                                                 
 multi_category_encoding (M  (None, 1776)              0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 1776)              3553      
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                56864     
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [5]:
# Load training dataset
df1 = pd.read_csv('../train.csv')
#df1.describe() # Visualize

# Load testing dataset
kaggle_test = pd.read_csv('../test.csv')
# test.describe() # Visualize

# Separate labels from training dataset for fitting
labels = df1['Activity']
del df1['Activity']

# 80/20 train test split
x_train, x_test, y_train, y_test = train_test_split(df1, labels, test_size=0.2)
input_shape = x_train.shape
print("Shape of Input Data: " + str(input_shape))
print("Shape of Test Labels: " + str(y_test.shape))

Shape of Input Data: (3000, 1776)
Shape of Test Labels: (751,)


In [6]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)

history = model.fit(
    x_train, y_train, epochs=100, validation_data=(x_test, y_test)
)

Epoch 1/100
94/94 [==============================] - 162s 1s/step - loss: 6.8371 - accuracy: 0.8077 - val_loss: 4.6604 - val_accuracy: 0.8083
Epoch 2/100
59/94 [=================>............] - ETA: 52s - loss: 4.4088 - accuracy: 0.8077

In [ ]:
prob_output = model.predict(kaggle_test)
print(prob_output)
print(prob_output.shape)
print(kaggle_test.shape)
np.savetxt("my_data.csv", prob_output, delimiter=",")